In [1]:
import logging
from datetime import datetime
import os

LOG_DIR = "logs"

def get_log_file_name():
    return f"log_{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}.log"

LOG_FILE_NAME = get_log_file_name()

os.makedirs(LOG_DIR, exist_ok=True)

LOG_FILE_PATH = os.path.join(LOG_DIR, LOG_FILE_NAME)

LOG_FILE_PATH

'logs\\log_2024-08-07-01-18-54.log'

In [4]:
import pandas as pd

In [9]:
chunk_df = pd.read_csv(r'titanic.csv', chunksize=10)
chunk_df

In [17]:
next(chunk_df)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
60,61,0,3,"Sirayanian, Mr. Orsen",male,22.0,0,0,2669,7.2292,NaN,C
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0000,B28,NaN
62,63,0,1,"Harris, Mr. Henry Birkhardt",male,45.0,1,0,36973,83.4750,C83,S
63,64,0,3,"Skoog, Master. Harald",male,4.0,3,2,347088,27.9000,NaN,S
64,65,0,1,"Stewart, Mr. Albert A",male,NaN,0,0,PC 17605,27.7208,NaN,C
65,66,1,3,"Moubarek, Master. Gerios",male,NaN,1,1,2661,15.2458,NaN,C
66,67,1,2,"Nye, Mrs. (Elizabeth Ramell)",female,29.0,0,0,C.A. 29395,10.5000,F33,S
67,68,0,3,"Crease, Mr. Ernest James",male,19.0,0,0,S.P. 3464,8.1583,NaN,S
68,69,1,3,"Andersson, Miss. Erna Alexandra",female,17.0,4,2,3101281,7.9250,NaN,S
69,70,0,3,"Kink, Mr. Vincenz",male,26.0,2,0,315151,8.6625,NaN,S


In [23]:
df = pd.read_csv(r'titanic.csv')
df.shape

(891, 12)

In [24]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [26]:
df.values[0, :]

array([1, 0, 3, 'Braund, Mr. Owen Harris', 'male', 22.0, 1, 0,
       'A/5 21171', 7.25, nan, 'S'], dtype=object)

In [31]:
dict(zip(df.columns, df.values[0, :]))

{'PassengerId': 1,
 'Survived': 0,
 'Pclass': 3,
 'Name': 'Braund, Mr. Owen Harris',
 'Sex': 'male',
 'Age': 22.0,
 'SibSp': 1,
 'Parch': 0,
 'Ticket': 'A/5 21171',
 'Fare': 7.25,
 'Cabin': nan,
 'Embarked': 'S'}

In [6]:
import json
import pandas as pd

class Generic:
    
    def __init__(self, record: dict) -> None:
        for k, v in record.items():
            setattr(self, k, v)
    
    @staticmethod
    def dict_to_object(data: dict):
        return Generic(data)
    
    def to_dict(self):
        return self.__dict__
    
    @classmethod
    def get_object(cls, file_path):
        chunk_df = pd.read_csv(file_path, chunksize=10)
        
        for df in chunk_df:
            for data in df.values:
                generic = Generic(dict(zip(df.columns, data)))
                yield generic
                
    @classmethod
    def export_schema_to_create_confluent_schema(cls, file_path):
        columns = next(pd.read_csv(file_path, chunksize=10)).columns

        schema = {
            "type": "record",
            "namespace": "com.mycorp.mynamespace",
            "name": "sampleRecord",
            "doc": "Sample schema to help you get started.",
            "fields": []
        }
        
        for column in columns:
            schema["fields"].append({
                "name": column,
                "type": "string",
                "doc": "The string type."
            })

        with open("car_sensor.json", "w") as f:
            json.dump(schema, f)

        schema_json = json.dumps(schema)
        
        return schema_json
    
    def __str__(self) -> str:
        return f"{self.__dict__}"
    

In [56]:
generic = Generic({'name': 'Bapan', 'city': 'Bangalore'})
print(generic)

{'name': 'Bapan', 'city': 'Bangalore'}


In [7]:
schema = Generic.export_schema_to_create_confluent_schema(file_path=r'sample_data/kafka-sensor-topic/aps_failure_training_set1.csv')
schema

'{"type": "record", "namespace": "com.mycorp.mynamespace", "name": "sampleRecord", "doc": "Sample schema to help you get started.", "fields": [{"name": "class", "type": "string", "doc": "The string type."}, {"name": "aa_000", "type": "string", "doc": "The string type."}, {"name": "ab_000", "type": "string", "doc": "The string type."}, {"name": "ac_000", "type": "string", "doc": "The string type."}, {"name": "ad_000", "type": "string", "doc": "The string type."}, {"name": "ae_000", "type": "string", "doc": "The string type."}, {"name": "af_000", "type": "string", "doc": "The string type."}, {"name": "ag_000", "type": "string", "doc": "The string type."}, {"name": "ag_001", "type": "string", "doc": "The string type."}, {"name": "ag_002", "type": "string", "doc": "The string type."}, {"name": "ag_003", "type": "string", "doc": "The string type."}, {"name": "ag_004", "type": "string", "doc": "The string type."}, {"name": "ag_005", "type": "string", "doc": "The string type."}, {"name": "ag_

In [44]:
# chunk_df = pd.read_csv(r'titanic.csv', chunksize=10)

# for df in chunk_df:
#     for data in df.values:
#         generic = Generic(dict(zip(df.columns, data)))
#         print(generic.to_dict())
#         print('===================================================================================\n')

In [33]:
def generate_data():
    chunk_df = pd.read_csv(r'titanic.csv', chunksize=10)

    for df in chunk_df:
        for data in df.values:
            generic = Generic(dict(zip(df.columns, data)))
            yield generic

gen = generate_data()
type(gen)

generator

In [47]:
columns = next(pd.read_csv(f'titanic.csv', chunksize=10)).columns
columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [43]:
next(gen).to_dict()

{'PassengerId': 9,
 'Survived': 1,
 'Pclass': 3,
 'Name': 'Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)',
 'Sex': 'female',
 'Age': 27.0,
 'SibSp': 0,
 'Parch': 2,
 'Ticket': '347742',
 'Fare': 11.1333,
 'Cabin': nan,
 'Embarked': 'S'}

In [22]:
# i = 0
# for df in chunk_df:
#     print(i, '========================================')
#     print(df)
#     i += 1

In [50]:
record1 = {"name": "Bapan Bairagya", "city": "Bangalore", "salary": 1000}
generic = Generic(record1)

generic.salary

1000

In [55]:
print(generic)

In [3]:
generic.to_dict()

{'name': 'Bapan Bairagya', 'city': 'Bangalore', 'salary': 1000}

In [4]:
generic.age

AttributeError: 'Generic' object has no attribute 'age'

In [5]:
record2 = {"name": "Purnima", "age": 50, "city": "Bangalore", "hobbies": "Cooking"}
generic1 = Generic(record2)

generic1.name

'Purnima'

In [6]:
generic1.to_dict()

{'name': 'Purnima', 'age': 50, 'city': 'Bangalore', 'hobbies': 'Cooking'}

In [4]:
import os

SAMPLE_DIR = os.path.join("sample_data")
print(SAMPLE_DIR)

topics = os.listdir(SAMPLE_DIR)
print(f'topics: [{topics}]')

for topic in topics:
    sample_topic_data_dir = os.path.join(SAMPLE_DIR, topic)
    print(sample_topic_data_dir)
    
    sample_file_path = os.path.join(sample_topic_data_dir, os.listdir(sample_topic_data_dir)[0])
    print(sample_file_path)

sample_data
topics: [['kafka-sensor-topic']]
sample_data\kafka-sensor-topic
sample_data\kafka-sensor-topic\aps_failure_training_set1.csv


In [5]:
sensor_df = next(pd.read_csv(r'sample_data/kafka-sensor-topic/aps_failure_training_set1.csv', chunksize=10))

#sensor_df.to_csv('sensor_for_avro_schema.csv', index=False)

In [6]:
sensor_df.dtypes

class     object
aa_000     int64
ab_000    object
ac_000    object
ad_000    object
           ...  
ee_007    object
ee_008    object
ee_009    object
ef_000    object
eg_000    object
Length: 171, dtype: object

In [7]:
df = pd.read_csv('sample_data/car_sensor/aps_failure_training_set1.csv')
df.head(3)

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,pos,153204,0,182,na,0,0,0,0,0,...,129862,26872,34044,22472,34362,0,0,0,0,0
1,pos,453236,na,2926,na,0,0,0,0,222,...,7908038,3026002,5025350,2025766,1160638,533834,493800,6914,0,0
2,pos,72504,na,1594,1052,0,0,0,244,178226,...,1432098,372252,527514,358274,332818,284178,3742,0,0,0


In [9]:
df.shape


(36188, 171)

In [9]:
df.isnull().sum()

class     0
aa_000    0
ab_000    0
ac_000    0
ad_000    0
         ..
ee_007    0
ee_008    0
ee_009    0
ef_000    0
eg_000    0
Length: 171, dtype: int64

In [10]:
df[df['ab_000'] == 'na']

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
1,pos,453236,na,2926,na,0,0,0,0,222,...,7908038,3026002,5025350,2025766,1160638,533834,493800,6914,0,0
2,pos,72504,na,1594,1052,0,0,0,244,178226,...,1432098,372252,527514,358274,332818,284178,3742,0,0,0
3,pos,762958,na,na,na,na,na,776,281128,2186308,...,na,na,na,na,na,na,na,na,na,na
4,pos,695994,na,na,na,na,na,0,0,0,...,1397742,495544,361646,28610,5130,212,0,0,na,na
7,pos,791254,na,na,na,na,na,0,14868,921256,...,3625532,1380864,3262488,5727988,7563752,12114820,264714,198,na,na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36179,neg,39308,na,452,374,0,0,0,0,0,...,312426,161694,323438,292350,250112,157636,234932,9506,0,0
36183,neg,153002,na,664,186,0,0,0,0,0,...,998500,566884,1290398,1218244,1019768,717762,898642,28588,0,0
36184,neg,2286,na,2130706538,224,0,0,0,0,0,...,10578,6760,21126,68424,136,0,0,0,0,0
36186,neg,80292,na,2130706432,494,0,0,0,0,0,...,699352,222654,347378,225724,194440,165070,802280,388422,0,0


In [8]:
sensor_df.columns

Index(['class', 'aa_000', 'ab_000', 'ac_000', 'ad_000', 'ae_000', 'af_000',
       'ag_000', 'ag_001', 'ag_002',
       ...
       'ee_002', 'ee_003', 'ee_004', 'ee_005', 'ee_006', 'ee_007', 'ee_008',
       'ee_009', 'ef_000', 'eg_000'],
      dtype='object', length=171)

In [3]:
import json
import pandas as pd # type: ignore


class SensorData:
    
    def __init__(self, record: dict) -> None:
        for k, v in record.items():
            setattr(self, k, v)
    
    @staticmethod
    def dict_to_object(data: dict):
        return SensorData(data)
    
    def to_dict(self):
        return self.__dict__
    
    @classmethod
    def get_object(cls, file_path):
        chunk_df = pd.read_csv(file_path, chunksize=10)
        
        for df in chunk_df:
            for data in df.values:
                generic = SensorData(dict(zip(df.columns, data)))
                yield generic
    
    @classmethod
    def generate_avro_schema(cls, file_path, record_name="SensorRecord", namespace="com.mycorp.mynamespace"):
        df = next(pd.read_csv(file_path, chunksize=10))
        
        # Start building the schema
        schema = {
            "type": "record",
            "namespace": namespace,
            "name": record_name,
            "fields": []
        }

        # Infer the schema for each column
        for column in df.columns:
            field_type = ["null", "string"]  # Default to string type, considering "na" and other non-numeric values
            field_schema = {
                "name": column,
                "type": field_type,
                "default": None
            }
            schema["fields"].append(field_schema)
        
            # Save the schema to a .avsc file
        schema_file_path = file_path.replace('.csv', '.avsc')
        with open(schema_file_path, 'w') as schema_file:
            json.dump(schema, schema_file, indent=2)

        print(f"Avro schema saved to {schema_file_path}")
        
        return schema

In [4]:
avro_schema = SensorData.generate_avro_schema('sample_data/kafka-sensor-topic/aps_failure_training_set1.csv')

Avro schema saved to sample_data/kafka-sensor-topic/aps_failure_training_set1.avsc


In [6]:
# avro_schema

In [11]:
import json
import pandas as pd


class SensorData:
    
    def __init__(self, record: dict) -> None:
        for k, v in record.items():
            setattr(self, k, v)
    
    @staticmethod
    def dict_to_object(data: dict):
        return SensorData(data)
    
    def to_dict(self):
        return self.__dict__
    
    def clean_data(self):
        """
        Cleans the data by replacing None or empty string values with a default value.
        """
        cleaned_data = {}
        for key, value in self.to_dict().items():
            if value is None or value == 'na':
                cleaned_data[key] = None  # Replace with an empty string or a default value (None)
            else:
                cleaned_data[key] = value
        return cleaned_data
    
    @classmethod
    def get_object(cls, file_path):
        chunk_df = pd.read_csv(file_path, chunksize=10, encoding='utf-8')
        
        for df in chunk_df:
            for data in df.values:
                # Convert to strings and handle 'na' as None
                cleaned_data = {}
                for k, v in zip(df.columns, data):
                    cleaned_data[k] = 'null' if v == 'na' else str(v)
                
                generic = SensorData(cleaned_data)
                yield generic.to_dict()
    
    @classmethod
    def generate_avro_schema(cls, file_path, record_name="SensorRecord", namespace="com.mycorp.mynamespace"):
        df = next(pd.read_csv(file_path, chunksize=10))
        
        # Start building the schema
        schema = {
            "type": "record",
            "namespace": namespace,
            "name": record_name,
            "fields": []
        }

        # Infer the schema for each column
        for column in df.columns:
            field_type = ["null", "string"]  # Default to string type, considering "na" and other non-numeric values
            field_schema = {
                "name": column,
                "type": field_type,
                "default": None
            }
            schema["fields"].append(field_schema)
        
            # Save the schema to a .avsc file
        schema_file_path = file_path.replace('.csv', '.avsc')
        with open(schema_file_path, 'w') as schema_file:
            json.dump(schema, schema_file, indent=2)

        print(f"Avro schema saved to {schema_file_path}")
        
        return schema

In [12]:
df.head()

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,pos,153204,0,182,na,0,0,0,0,0,...,129862,26872,34044,22472,34362,0,0,0,0,0
1,pos,453236,na,2926,na,0,0,0,0,222,...,7908038,3026002,5025350,2025766,1160638,533834,493800,6914,0,0
2,pos,72504,na,1594,1052,0,0,0,244,178226,...,1432098,372252,527514,358274,332818,284178,3742,0,0,0
3,pos,762958,na,na,na,na,na,776,281128,2186308,...,na,na,na,na,na,na,na,na,na,na
4,pos,695994,na,na,na,na,na,0,0,0,...,1397742,495544,361646,28610,5130,212,0,0,na,na


In [17]:
next(SensorData.get_object('sample_data/car_sensor_topic/aps_failure_training_set1.csv'))

{'class': 'pos',
 'aa_000': '153204',
 'ab_000': '0',
 'ac_000': '182',
 'ad_000': 'null',
 'ae_000': '0',
 'af_000': '0',
 'ag_000': '0',
 'ag_001': '0',
 'ag_002': '0',
 'ag_003': '11804',
 'ag_004': '684444',
 'ag_005': '326536',
 'ag_006': '31586',
 'ag_007': '0',
 'ag_008': '0',
 'ag_009': '0',
 'ah_000': '2658638',
 'ai_000': '14346',
 'aj_000': '0',
 'ak_000': '0',
 'al_000': '29384',
 'am_0': '46356',
 'an_000': '5566182',
 'ao_000': '4426834',
 'ap_000': '1571480',
 'aq_000': '1120644',
 'ar_000': '2',
 'as_000': '0',
 'at_000': '0',
 'au_000': '0',
 'av_000': '1368',
 'ax_000': '862',
 'ay_000': '0',
 'ay_001': '0',
 'ay_002': '0',
 'ay_003': '0',
 'ay_004': '0',
 'ay_005': '0',
 'ay_006': '0',
 'ay_007': '94188',
 'ay_008': '960182',
 'ay_009': '0',
 'az_000': '250',
 'az_001': '16',
 'az_002': '32',
 'az_003': '146',
 'az_004': '126',
 'az_005': '951126',
 'az_006': '91162',
 'az_007': '11512',
 'az_008': '0',
 'az_009': '0',
 'ba_000': '702994',
 'ba_001': '186172',
 'ba_0

In [18]:
from src.constant import SAMPLE_DIR

SAMPLE_DIR

'sample_data'

In [22]:
import os

topics = os.listdir(SAMPLE_DIR)
print(f'topics: [{topics}]')

for topic in topics:
    sample_topic_data_dir = os.path.join(SAMPLE_DIR, topic)
    print(sample_topic_data_dir)
    sample_file_path = os.path.join(sample_topic_data_dir, os.listdir(sample_topic_data_dir)[0])
    print(sample_file_path)

topics: [['car_sensor']]
sample_data\car_sensor
sample_data\car_sensor\aps_failure_training_set1.csv


In [2]:
import os
from dotenv import load_dotenv
from pymongo import MongoClient

# Load environment variables from .env file
load_dotenv()

# Connection string
CONN_STRING = os.getenv('MONGODB_CONN_STRING')

# Connect to MongoDB
client = MongoClient(CONN_STRING)

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [3]:
# Select the database
db = client['sensor_db']

# Select the collection
collection = db['car_sensor']

In [8]:
# Query the collection

find_query = {"aa_000": {"$gt": "70000"}}

count_results = collection.count_documents(find_query)
print("Total Documents Found : ",count_results)

Total Documents Found :  79
